# 6. Proposition d'amélioration

Dans cette partie, nous allons proposer des améliorations au conditions de travail des employés pour résoudre le problème de HumanForYou.

Nous allons chercher a modifier les conditions de travail dans le sens de nos trouvailles graves aux résultats précédents.

Nous avons découvert grâce au calcule de la variation des valeurs du model `MultiLayeredPercepton` que les trois facons de diminuer le temps de travail moyen et augmenter le temps de formation de l'année passée.

Nous allons donc modifier ces variables dans leurs sens respectifs pour esperer en tirer un taux d'attretion faible

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore')

md = pd.read_excel("./buffer/1-main_dataset.xlsx")
md.drop(columns=md.columns[0], axis=1, inplace=True)

X = md.drop(columns="Attrition", axis=1)
y = [i == "Yes" for i in md["Attrition"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,  random_state=0)


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_dict = {
    'Random Forest': RandomForestClassifier(random_state=42, class_weight={0: 1, 1: 5}, max_depth=30, min_samples_leaf= 3, min_samples_split=2, n_estimators=130),
    'Logistic Regression': LogisticRegression(C=0.15, penalty="l2"),
    'Multi Layered Perceptron': MLPClassifier(random_state=42, alpha=0.1, hidden_layer_sizes=100, max_iter=1600, solver='lbfgs')
}

# Get the list of columns depending on there type
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='median')),
      ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
      ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[
      ('num', numeric_transformer, numeric_features),
      ('cat', categorical_transformer, categorical_features)])

full_pipeline_dict = {}
for name, model in model_dict.items():
      full_pipeline_dict[name] = Pipeline(steps=[
            ('preprocessor', preprocessor),
            (name, model)])

In [3]:
y_pred_dict = {}

for name, pipeline in full_pipeline_dict.items():
    pipeline.fit(X_train, y_train)
    score = pipeline.score(X_test, y_test)
    y_pred_dict[name] = pipeline.predict(X_test)
    print(f"{name} score: {score}")

Random Forest score: 0.9807256235827665
Logistic Regression score: 0.8503401360544217
Multi Layered Perceptron score: 0.9909297052154195


In [6]:
offset_dict = {
    'Average': -0.2,
    'TrainingTimesLastYear': 2
}

X_test_mod = X_test.copy()

for key, val in offset_dict.items():
    X_test_mod[key] += np.array( [val] * len(X_test_mod[key]))

y_pred_mod_dict = {}
for name, pipeline in full_pipeline_dict.items():
    y_pred_mod_dict[name] = pipeline.predict(X_test_mod)

print(f"Real Attrition: {sum(y_test)}\nPredicted Attrition: { {k:sum(v) for k, v in y_pred_dict.items()} }\nImproved Attrition: { {k:sum(v) for k, v in y_pred_mod_dict.items()} }")

Real Attrition: 145
Predicted Attrition: {'Random Forest': 128, 'Logistic Regression': 59, 'Multi Layered Perceptron': 143}
Improved Attrition: {'Random Forest': 45, 'Logistic Regression': 0, 'Multi Layered Perceptron': 3}


## Conclusion

Graces aux recherches effectuées précédament et au models développés à partir des données des employées, nous avons pue proposer une solution à HumanForYou pour diminuer leur taux d'attertion.

Cette solution est étique car elle ne cause aucune discrimination et permet juste de modifier les confditions de travail pour arriver a de meilleurs résultats.

Si cette solution ne convient pas a HumanForYou, les models et données éxtraitres permetterons surement aux personnes concerner de trouver un comprmis. De plus, il sera facile pour eux de tester leurs solution avec notre model en modifiant le dictionaire `offset_dict`. Il sera par contre demander de prendre en compte l'étique lors de cette opération.